![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.5.Clinical_Deidentification_Utility_Module.ipynb)

# Setup

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install --upgrade -q spark-nlp-display

In [3]:
import json
import os
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())


spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 4.3.2
Spark NLP_JSL Version : 4.3.2


# Module

Description of Parameters: <br/>


---



---


`custom_pipeline` : Sparknlp PipelineModel, optional
            custom PipelineModel to be used for deidentification, by default None <br/>
        `ner_chunk` : str, optional
            final chunk column name of custom pipeline that will be deidentified, by default "ner_chunk" <br/>
        `fields` : dict, optional
            fields to be deidentified and their deidentification modes, by default {"text": "mask"} <br/>
        `sentence` : str, optional
            sentence column name of the given custom pipeline, by default "sentence" <br/>
        `token` : str, optional
            token column name of the given custom pipeline, by default "token" <br/>
        `document` : str, optional
            document column name of the given custom pipeline, by default "document" <br/>
        `masking_policy` : str, optional
            masking policy, by default "entity_labels" <br/>
        `fixed_mask_length` : int, optional
            fixed mask length, by default 4 <br/>
        `obfuscate_date` : bool, optional
            obfuscate date, by default True <br/>
        `obfuscate_ref_source` : str, optional
            obfuscate reference source, by default "faker" <br/>
        `obfuscate_ref_file_path` : str, optional
            obfuscate reference file path, by default None <br/>
        `age_group_obfuscation` : bool, optional
            age group obfuscation, by default False <br/>
        `age_ranges` : list, optional
            age ranges for obfuscation, by default [1, 4, 12, 20, 40, 60, 80] <br/>
        `shift_days` : bool, optional
            shift days, by default False <br/>
        `number_of_days` : int, optional
            number of days, by default None <br/>
        `documentHashCoder_col_name` : str, optional
            document hash coder column name, by default "documentHash" <br/>
        `date_tag` : str, optional
            date tag, by default "DATE" <br/>
        `language` : str, optional
            language, by default "en" <br/>
        `region` : str, optional
            region, by default "us" <br/>
        `unnormalized_date` : bool, optional
            unnormalized date, by default False <br/>
        `unnormalized_mode` : str, optional
            unnormalized mode, by default "mask" <br/>
        `id_column_name` : str, optional
            ID column name, by default "id" <br/>
        `date_shift_column_name` : str, optional
            date shift column name, by default "date_shift" <br/>
        `separator` : str, optional
            separator of input csv file, by default "\t" <br/> 
        `input_file_path` : str, optional
            input file path, by default None <br/>
        `output_file_path` : str, optional
            output file path, by default 'deidentified.csv'

Returns


---

Spark DataFrame: Spark DataFrame with deidentified text <br/>
csv/json file: A deidentified file.

In [4]:
#sample data
text ='''
Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 . PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .
'''

df= spark.createDataFrame([[text]]).toDF("text")

In [5]:
df_pd= df.toPandas()
df_pd.to_csv("deid_data.csv", index=False)

# With a custom pipeline

Sample custom pipeline with `ner_deid_generic_augmented` to detect PHI entities. 

In [6]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

deid_ner = MedicalNerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setWhiteList(['NAME', 'PROFESSION', 'ID', 'AGE', 'DATE'])

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model= nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_generic_augmented download started this may take some time.
[OK!]


## Default parameters

In [7]:
from sparknlp_jsl import Deid

In [8]:
# we need to feed the module with an active spark session
deid_implementor= Deid(spark)

In [9]:
res= deid_implementor.deidentify(input_file_path="deid_data.csv",
                                 output_file_path="deidentified.csv", 
                                 custom_pipeline=model)

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [10]:
res.show(truncate=70)

+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                                  text|                                                     text_deidentified|
+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|                                                                     "|                                                                     "|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : <DATE> , <NAME> , M.D . , Name : <NAME> , MR # <ID> D...|
|  2|                                                                     "|                                                                     "|
+---+----------------------------------------------------------------------+------------------------------------

In [11]:
#checking saved output file
import pandas as pd
res_data= pd.read_csv("deidentified.csv")
res_data.head()

,ID,text,text_deidentified
0,0,"""",""""
1,1,"Record date : 2093-01-13 , David Hale , M.D . ...","Record date : <DATE> , <NAME> , M.D . , Name :..."
2,2,"""",""""


## Mask options 


### same_length_chars

In [12]:
res= deid_implementor.deidentify(input_file_path="deid_data.csv",
                                output_file_path="deidentified.csv",
                                custom_pipeline=model,
                                fields={"text": "mask"}, masking_policy="same_length_chars")

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [13]:
res.show(truncate=70)

+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                                  text|                                                     text_deidentified|
+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|                                                                     "|                                                                     "|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : [********] , [********] , M.D . , Name : [***********...|
|  2|                                                                     "|                                                                     "|
+---+----------------------------------------------------------------------+------------------------------------

### fixed_length_chars

In [14]:
res= deid_implementor.deidentify(input_file_path="deid_data.csv",
                                output_file_path="deidentified.csv",
                                custom_pipeline=model,
                                fields={"text": "mask"}, masking_policy="fixed_length_chars", fixed_mask_length=2)

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [15]:
res.show(truncate=70)

+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                                  text|                                                     text_deidentified|
+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|                                                                     "|                                                                     "|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : ** , ** , M.D . , Name : ** , MR # ** Date : ** . PCP...|
|  2|                                                                     "|                                                                     "|
+---+----------------------------------------------------------------------+------------------------------------

### masking multiple columns

In [16]:
text= """Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR ."""
text_1= """Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 """
df= spark.createDataFrame([[text, text_1]]).toDF("text", "text_1")

df_pd= df.toPandas()
df_pd.to_csv("deid_multiple_data.csv", index=False)
df_pd.head()

,text,text_1
0,"Record date : 2093-01-13 , David Hale , M.D . ...","Date : 01/13/93 PCP : Oliveira , 25 years-old ..."


In [17]:
res= deid_implementor.deidentify(input_file_path="deid_multiple_data.csv",
                                output_file_path="deidentified.csv",
                                custom_pipeline=model,
                                fields={"text": "mask", "text_1":"obfuscate"}, masking_policy="fixed_length_chars", 
                                fixed_mask_length=2, separator=",")

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentification process of the 'text_1' field has begun...
Deidentification process of the 'text_1' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [18]:
res.show(truncate=70)

+---+----------------------------------------------------------------------+----------------------------------------------+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                                  text|                             text_deidentified|                                                                text_1|                                                   text_1_deidentified|
+---+----------------------------------------------------------------------+----------------------------------------------+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : ** , ** , M.D . , Name : ** MR .|Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-...|Date : 0

## Obfuscate Options

### obfuscate_ref_source="file"

In [19]:
obs_lines = """Marvin MARSHALL#NAME
Hubert GROGAN#NAME
ALTHEA COLBURN#NAME
Kalil AMIN#NAME
Inci FOUNTAIN#NAME
Ekaterina Rosa#DOCTOR
Rudiger Chao#DOCTOR
COLLETTE KOHLER#DOCTOR
Mufi HIGGS#DOCTOR"""


with open ('obfuscation.txt', 'w') as f:
  f.write(obs_lines)

In [20]:
text= """Name of the patient is Leah Shannon. 
Her pps number is 1234567. 
"""

df= spark.createDataFrame([[text]]).toDF("text")
df_pd= df.toPandas()
df_pd.to_csv("deid_obfs_data.csv", index=False)

In [21]:
res= deid_implementor.deidentify(input_file_path="deid_obfs_data.csv",
                                output_file_path="deidentified.csv",
                                custom_pipeline=model,
                                fields={"text": "obfuscate"}, obfuscate_ref_source="file",
                                obfuscate_ref_file_path="obfuscation.txt")

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [22]:
res.show(truncate=False)

+---+-------------------------------------+-------------------------------------+
|ID |text                                 |text_deidentified                    |
+---+-------------------------------------+-------------------------------------+
|0  |Name of the patient is Leah Shannon. |Name of the patient is Hubert GROGAN.|
|1  |Her pps number is 1234567.           |Her pps number is <ID>.              |
|2  |"                                    |"                                    |
+---+-------------------------------------+-------------------------------------+



### obfuscate_ref_source=both

In [23]:
res= deid_implementor.deidentify(input_file_path="deid_obfs_data.csv",
                                output_file_path="deidentified.csv",
                                custom_pipeline=model,
                                fields={"text": "obfuscate"}, obfuscate_ref_source="both",
                                obfuscate_ref_file_path="obfuscation.txt")

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [24]:
res.show(truncate=70)

+---+-------------------------------------+-------------------------------------+
| ID|                                 text|                    text_deidentified|
+---+-------------------------------------+-------------------------------------+
|  0|Name of the patient is Leah Shannon. |Name of the patient is Devonda Goody.|
|  1|          Her pps number is 1234567. |          Her pps number is N6148098.|
|  2|                                    "|                                    "|
+---+-------------------------------------+-------------------------------------+



### obfuscate_ref_source=faker 

In [25]:
res= deid_implementor.deidentify(input_file_path="deid_data.csv",
                                output_file_path="deidentified.csv",
                                custom_pipeline=model, obfuscate_date=True,
                                fields={"text": "obfuscate"})


Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [26]:
res.show(truncate=70)

+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                                  text|                                                     text_deidentified|
+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|                                                                     "|                                                                     "|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : 2093-01-18 , Christain Romp , M.D . , Name : Will Det...|
|  2|                                                                     "|                                                                     "|
+---+----------------------------------------------------------------------+------------------------------------

### age groups obfuscation

In [27]:
# Example data
dates = [
'1 year old baby',
'4 year old kids',
'A 15 year old female with',
'Record date: 2093-01-13, Age: 25',
'Patient is 45 years-old',
'He is 65 years-old male'
]
from pyspark.sql.types import StringType
df_dates = spark.createDataFrame(dates,StringType()).toDF('text')

pd_df_dates= df_dates.toPandas()
pd_df_dates.to_csv("deid_age_group.csv", index=False)

In [28]:
res= deid_implementor.deidentify(input_file_path="deid_age_group.csv",
                                                output_file_path="deidentified.csv",
                                                custom_pipeline=model,
                                                fields={"text": "obfuscate"},
                                                age_group_obfuscation=True, age_ranges=[1, 4, 12, 20, 40, 60, 80])

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [29]:
res.show(truncate=False)

+---+--------------------------------+--------------------------------+
|ID |text                            |text_deidentified               |
+---+--------------------------------+--------------------------------+
|0  |1 year old baby                 |3 year old baby                 |
|1  |4 year old kids                 |6 year old kids                 |
|2  |A 15 year old female with       |A 13 year old female with       |
|3  |Record date: 2093-01-13, Age: 25|Record date: 2093-01-21, Age: 20|
|4  |Patient is 45 years-old         |Patient is 55 years-old         |
|5  |He is 65 years-old male         |He is 77 years-old male         |
+---+--------------------------------+--------------------------------+



### shifting days according to the ID column

In [30]:
import pandas as pd
data = pd.DataFrame(
    {'patientID' : ['A001', 'A001', 'A002', 'A002'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 02/28/2020', 
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ]
    }
)

my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate = False)

+---------+----------------------------------------+
|patientID|text                                    |
+---------+----------------------------------------+
|A001     |Chris Brown was discharged on 10/02/2022|
|A001     |Mark White was discharged on 02/28/2020 |
|A002     |John was discharged on 03/15/2022       |
|A002     |John Moore was discharged on 12/31/2022 |
+---------+----------------------------------------+



In [31]:
df_pd= my_input_df.toPandas()
df_pd.to_csv("deid_id_data.csv", index=False)

Custom pipeline with `DocumentHashCoder()`. 

In [32]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setPatientIdColumn("patientID")\
    .setRangeDays(100)\
    .setNewDateShift("shift_days")\
    .setSeed(100)

tokenizer = Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document2", "token"])\
    .setOutputCol("word_embeddings")

clinical_ner = MedicalNerModel\
    .pretrained("ner_deid_subentity_augmented", "en", "clinical/models")\
    .setInputCols(["document2","token", "word_embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk")

    
nlpPipeline = Pipeline().setStages([
    documentAssembler,
    documentHasher,
    tokenizer,
    embeddings,
    clinical_ner,
    ner_converter])

empty_data = spark.createDataFrame([["", ""]]).toDF("text", "patientID")

pipeline_model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_subentity_augmented download started this may take some time.
[OK!]


In [33]:
res= deid_implementor.deidentify(input_file_path="deid_id_data.csv",
                                                output_file_path="deidentified.csv",
                                                custom_pipeline=pipeline_model,
                                                fields={"text": "obfuscate"},
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                ner_chunk="ner_chunk",
                                                token="token",
                                                documentHashCoder_col_name="document2",
                                                separator=",",
                                                unnormalized_date=False
                                                )

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [34]:
res.show(truncate=False)

+---+----------------------------------------+-----------------------------------------+
|ID |text                                    |text_deidentified                        |
+---+----------------------------------------+-----------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Mike Rosser was discharged on 09/27/2022 |
|1  |Mark White was discharged on 02/28/2020 |Jiles Garter was discharged on 02/23/2020|
|2  |John was discharged on 03/15/2022       |Gene Power was discharged on 04/13/2022  |
|3  |John Moore was discharged on 12/31/2022 |Reyna Balder was discharged on 01/29/2023|
+---+----------------------------------------+-----------------------------------------+



### shifting days according to specified values

In [35]:
data = pd.DataFrame(
    {'patientID' : ['A001', 'A002', 'A003', 'A004'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 03/01/2020', 
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ],
     'dateshift' : ['10', '-2', '30', '-8']
    }
)

my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate=False)

+---------+----------------------------------------+---------+
|patientID|text                                    |dateshift|
+---------+----------------------------------------+---------+
|A001     |Chris Brown was discharged on 10/02/2022|10       |
|A002     |Mark White was discharged on 03/01/2020 |-2       |
|A003     |John was discharged on 03/15/2022       |30       |
|A004     |John Moore was discharged on 12/31/2022 |-8       |
+---------+----------------------------------------+---------+



In [36]:
df_pd= my_input_df.toPandas()
df_pd.to_csv("deid_specific_data.csv", index=False)

In [37]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\

tokenizer = Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document2", "token"])\
    .setOutputCol("word_embeddings")

clinical_ner = MedicalNerModel\
    .pretrained("ner_deid_subentity_augmented", "en", "clinical/models")\
    .setInputCols(["document2","token", "word_embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline().setStages([
    documentAssembler,
    documentHasher,
    tokenizer,
    embeddings,
    clinical_ner,
    ner_converter
    
])

empty_data = spark.createDataFrame([["", "", ""]]).toDF("patientID","text", "dateshift")

pipeline_col_model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_subentity_augmented download started this may take some time.
[OK!]


In [38]:
res= deid_implementor.deidentify(input_file_path="deid_specific_data.csv",
                                                separator=",",
                                                output_file_path="deidentified.csv",
                                                custom_pipeline=pipeline_col_model,
                                                fields={"text": "obfuscate"},
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                ner_chunk="ner_chunk",
                                                token="token",
                                                documentHashCoder_col_name="document2")

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [39]:
res.show(truncate=False)

+---+----------------------------------------+---------------------------------------------+
|ID |text                                    |text_deidentified                            |
+---+----------------------------------------+---------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Noble Charters was discharged on 10/12/2022  |
|1  |Mark White was discharged on 03/01/2020 |Justo Jenkins was discharged on 02/28/2020   |
|2  |John was discharged on 03/15/2022       |Erlinda Paschal was discharged on 04/14/2022 |
|3  |John Moore was discharged on 12/31/2022 |Wendelin Harvard was discharged on 12/23/2022|
+---+----------------------------------------+---------------------------------------------+



### unnormalized date formats

In [40]:
import pandas as pd

data = pd.DataFrame(
    {'patientID' : ['A001', 'A001', 'A002', 'A002'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 02/28/2020', 
               'John was discharged on 03Apr2022',          # check this
               'John Moore was discharged on 12/31/2022'
              ],
     'dateshift' : ['-5', '-2', '10', '20']
    }
)


my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate=False)

+---------+----------------------------------------+---------+
|patientID|text                                    |dateshift|
+---------+----------------------------------------+---------+
|A001     |Chris Brown was discharged on 10/02/2022|-5       |
|A001     |Mark White was discharged on 02/28/2020 |-2       |
|A002     |John was discharged on 03Apr2022        |10       |
|A002     |John Moore was discharged on 12/31/2022 |20       |
+---------+----------------------------------------+---------+



In [41]:
df_pd= my_input_df.toPandas()
df_pd.to_csv("deid_unnormalized_data.csv", index=False)

In [42]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\

tokenizer = Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document2", "token"])\
    .setOutputCol("word_embeddings")

clinical_ner = MedicalNerModel\
    .pretrained("ner_deid_subentity_augmented", "en", "clinical/models")\
    .setInputCols(["document2","token", "word_embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline().setStages([
    documentAssembler,
    documentHasher,
    tokenizer,
    embeddings,
    clinical_ner,
    ner_converter    
])

empty_data = spark.createDataFrame([["", "", ""]]).toDF("patientID","text", "dateshift")

model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_subentity_augmented download started this may take some time.
[OK!]


In [43]:
res= deid_implementor.deidentify(input_file_path="deid_unnormalized_data.csv",
                                                output_file_path="deidentified.csv",
                                                custom_pipeline=model,
                                                fields={"text": "obfuscate"},
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                ner_chunk="ner_chunk",
                                                token="token",
                                                documentHashCoder_col_name="document2",
                                                separator=",",
                                                unnormalized_date=True,
                                                unnormalized_mode="mask"
                                                )

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [44]:
res.show(truncate=False)

+---+----------------------------------------+-------------------------------------------+
|ID |text                                    |text_deidentified                          |
+---+----------------------------------------+-------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Cristal Berber was discharged on 09/27/2022|
|1  |Mark White was discharged on 02/28/2020 |Antonina Spine was discharged on 02/26/2020|
|2  |John was discharged on 03Apr2022        |Nathaniel Genta was discharged on <DATE>   |
|3  |John Moore was discharged on 12/31/2022 |Suzzette Morel was discharged on 01/20/2023|
+---+----------------------------------------+-------------------------------------------+



**unnormalized_mode="obfuscate"**

In [45]:
res= deid_implementor.deidentify(input_file_path="deid_unnormalized_data.csv",
                                                output_file_path="deidentified1.csv",
                                                custom_pipeline=model,
                                                fields={"text": "obfuscate"},
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                ner_chunk="ner_chunk",
                                                token="token",
                                                documentHashCoder_col_name="document2",
                                                separator=",",
                                                unnormalized_date=True,
                                                unnormalized_mode="obfuscate"
                                                )

Deidentification process of the 'text' field has begun...
Deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [46]:
res.show(truncate=False)

+---+----------------------------------------+---------------------------------------------+
|ID |text                                    |text_deidentified                            |
+---+----------------------------------------+---------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Lucrecia Bollman was discharged on 09/27/2022|
|1  |Mark White was discharged on 02/28/2020 |Laurene Hobby was discharged on 02/26/2020   |
|2  |John was discharged on 03Apr2022        |Henrene Pardon was discharged on 07-04-1983  |
|3  |John Moore was discharged on 12/31/2022 |Berta Shake was discharged on 01/20/2023     |
+---+----------------------------------------+---------------------------------------------+



# With no custom pipeline

### Default parameters

In [47]:
res= deid_implementor.deidentify(input_file_path="deid_data.csv",
                                 output_file_path="deidentified.csv")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of the 'text' field has begun...
deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [48]:
res.show(truncate=70)

+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                         text_original|                                                             text_deid|
+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|                                                                     "|                                                                     "|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : <DATE> , <DOCTOR> , M.D . , Name : <PATIENT> , MR # <...|
|  2|                                                                     "|                                                                     "|
+---+----------------------------------------------------------------------+------------------------------------

### Mask options

#### entity_labels

In [49]:
res= deid_implementor.deidentify(input_file_path="deid_data.csv",
                                output_file_path="deidentified.csv",
                                fields={"text": "mask"}, masking_policy="entity_labels")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of the 'text' field has begun...
deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [50]:
res.show(truncate=70)

+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                         text_original|                                                             text_deid|
+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|                                                                     "|                                                                     "|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : <DATE> , <DOCTOR> , M.D . , Name : <PATIENT> , MR # <...|
|  2|                                                                     "|                                                                     "|
+---+----------------------------------------------------------------------+------------------------------------

#### fixed_length_chars

In [51]:
res= deid_implementor.deidentify(input_file_path="deid_data.csv",
                                output_file_path="deidentified.csv",
                                fields={"text": "mask"},
                                masking_policy="fixed_length_chars",
                                fixed_mask_length=2)

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of the 'text' field has begun...
deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [52]:
res.show(truncate=70)

+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                         text_original|                                                             text_deid|
+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|                                                                     "|                                                                     "|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : ** , ** , M.D . , Name : ** , MR # ** Date : ** . PCP...|
|  2|                                                                     "|                                                                     "|
+---+----------------------------------------------------------------------+------------------------------------

#### same_length_chars

In [53]:
res= deid_implementor.deidentify(input_file_path="deid_data.csv",
                                output_file_path="deidentified.csv",
                                fields={"text": "mask"},
                                masking_policy="same_length_chars")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of the 'text' field has begun...
deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [54]:
res.show(truncate=70)

+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                         text_original|                                                             text_deid|
+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|                                                                     "|                                                                     "|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : [********] , [********] , M.D . , Name : [***********...|
|  2|                                                                     "|                                                                     "|
+---+----------------------------------------------------------------------+------------------------------------

### Obfuscate option

In [55]:
res= deid_implementor.deidentify(input_file_path="deid_data.csv",
                                output_file_path="deidentified.csv",
                                separator="\t",
                                fields={"text": "obfuscate"},
                                unnormalized_date=False)

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of the 'text' field has begun...
deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [56]:
res.show(truncate=70) 

+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
| ID|                                                         text_original|                                                             text_deid|
+---+----------------------------------------------------------------------+----------------------------------------------------------------------+
|  0|                                                                     "|                                                                     "|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|Record date : 2093-03-04 , Dr Pat Bodily , M.D . , Name : Shirleen ...|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|       PCP : Lamont Lineman , 36years-old , Record date : 2079-12-25 .|
|  1|Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson ...|RENOWN SOUTH MEADOWS MEDICAL CENTER 

#### age groups obfuscation

In [57]:
res= deid_implementor.deidentify(input_file_path="deid_age_group.csv",
                                                output_file_path="deidentified.csv",
                                                fields={"text": "obfuscate"},
                                                age_group_obfuscation=True, age_ranges=[1, 4, 12, 20, 40, 60, 80])

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of the 'text' field has begun...
deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [58]:
res.show(truncate=70)

+---+--------------------------------+--------------------------------+
| ID|                   text_original|                       text_deid|
+---+--------------------------------+--------------------------------+
|  0|                 1 year old baby|                  2year old baby|
|  1|                 4 year old kids|                  5year old kids|
|  2|       A 15 year old female with|        A 13year old female with|
|  3|Record date: 2093-01-13, Age: 25|Record date: 2093-03-14, Age: 36|
|  4|         Patient is 45 years-old|          Patient is 52years-old|
|  5|         He is 65 years-old male|          He is 62years-old male|
+---+--------------------------------+--------------------------------+



#### shifting days

In [59]:
data = pd.DataFrame(
    {'patientID' : ['A001', 'A002', 'A003', 'A004'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 03/01/2020', 
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ]
    }
)

my_input_df = spark.createDataFrame(data)
df_pd= my_input_df.toPandas()
df_pd.to_csv("shift_days_data.csv", index=False)

my_input_df.show(truncate=False)

+---------+----------------------------------------+
|patientID|text                                    |
+---------+----------------------------------------+
|A001     |Chris Brown was discharged on 10/02/2022|
|A002     |Mark White was discharged on 03/01/2020 |
|A003     |John was discharged on 03/15/2022       |
|A004     |John Moore was discharged on 12/31/2022 |
+---------+----------------------------------------+



In [60]:
res= deid_implementor.deidentify(input_file_path="shift_days_data.csv",
                                                output_file_path="deidentified.csv",
                                                fields={"text": "obfuscate"},
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                separator=",",
                                                number_of_days=2)

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of the 'text' field has begun...
deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [61]:
res.show(truncate=False)

+---+----------------------------------------+------------------------------------------+
|ID |text_original                           |text_deid                                 |
+---+----------------------------------------+------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Dotty Neat was discharged on 10/04/2022   |
|1  |Mark White was discharged on 03/01/2020 |Jennings Abt was discharged on 03/03/2020 |
|2  |John was discharged on 03/15/2022       |Bolling Kells was discharged on 03/17/2022|
|3  |John Moore was discharged on 12/31/2022 |Ellender Rom was discharged on 01/02/2023 |
+---+----------------------------------------+------------------------------------------+



#### unnormalized date formats

In [62]:
res= deid_implementor.deidentify(input_file_path="deid_unnormalized_data.csv",
                                                output_file_path="deidentified.csv",
                                                fields={"text": "obfuscate"},
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                separator=",",
                                                unnormalized_date=True,
                                                unnormalized_mode="obfuscate"
                                                )

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of the 'text' field has begun...
deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [63]:
res.show(truncate=False)

+---+----------------------------------------+------------------------------------------+
|ID |text_original                           |text_deid                                 |
+---+----------------------------------------+------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Dotty Neat was discharged on 10/04/2022   |
|1  |Mark White was discharged on 02/28/2020 |Jennings Abt was discharged on 03/01/2020 |
|2  |John was discharged on 03Apr2022        |Bolling Kells was discharged on 11-02-2001|
|3  |John Moore was discharged on 12/31/2022 |Ellender Rom was discharged on 01/02/2023 |
+---+----------------------------------------+------------------------------------------+



**unnormalized_mode="mask"**

In [64]:
res= deid_implementor.deidentify(input_file_path="deid_unnormalized_data.csv",
                                                output_file_path="deidentified.csv",
                                                fields={"text": "obfuscate"},
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                separator=",",
                                                unnormalized_date=True,
                                                unnormalized_mode="mask"
                                                )

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of the 'text' field has begun...
deidentification process of the 'text' field was completed...
Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [65]:
res.show(truncate=False)

+---+----------------------------------------+-----------------------------------------+
|ID |text_original                           |text_deid                                |
+---+----------------------------------------+-----------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Dotty Neat was discharged on 10/04/2022  |
|1  |Mark White was discharged on 02/28/2020 |Jennings Abt was discharged on 03/01/2020|
|2  |John was discharged on 03Apr2022        |Bolling Kells was discharged on <DATE>   |
|3  |John Moore was discharged on 12/31/2022 |Ellender Rom was discharged on 01/02/2023|
+---+----------------------------------------+-----------------------------------------+



# Structured Deidentification

In [66]:
#sample data
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/hipaa-table-001.txt

df = spark.read.format("csv") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("hipaa-table-001.txt")

df = df.withColumnRenamed("PATIENT","NAME")
df.show(truncate=False)

+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
|NAME           |DOB       |AGE|ADDRESS                                             |ZIPCODE|TEL           |SBP|DBP|
+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
|Cecilia Chapman|04/02/1935|83 |711-2880 Nulla St. Mankato Mississippi              |69200  |(257) 563-7401|101|42 |
|Iris Watson    |03/10/2009|9  |P.O. Box 283 8562 Fusce Rd. Frederick Nebraska      |20620  |(372) 587-2335|159|122|
|Bryar Pitts    |11/01/1921|98 |5543 Aliquet St. Fort Dodge GA                      |20783  |(717) 450-4729|149|52 |
|Theodore Lowe  |13/02/2002|16 |Ap #867-859 Sit Rd. Azusa New York                  |39531  |(793) 151-6230|134|115|
|Calista Wise   |20/08/1942|76 |7292 Dictum Av. San Antonio MI                      |47096  |(492) 709-6392|139|78 |
|Kyla Olsen     |12/05/1973|45 |Ap #651-8679 Sodales Av. Tamunin

In [67]:
df_pd= df.toPandas()
df_pd.to_csv("deid_structured_data.csv", index=False)

## Default parameters

In [68]:
res= deid_implementor.structured_deidentifier(input_file_path="deid_structured_data.csv")

Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [69]:
res.show()

+--------------------+----------+-----+--------------------+-------+--------------+---+---+
|                NAME|       DOB|  AGE|             ADDRESS|ZIPCODE|           TEL|SBP|DBP|
+--------------------+----------+-----+--------------------+-------+--------------+---+---+
|    [Myrlene Merlin]|04/02/1935| [87]|711-2880 Nulla St...|  69200|(257) 563-7401|101| 42|
|  [Katie Schlichter]|03/10/2009|  [7]|P.O. Box 283 8562...|  20620|(372) 587-2335|159|122|
|    [Lendon Handing]|11/01/1921| [94]|5543 Aliquet St. ...|  20783|(717) 450-4729|149| 52|
|       [Carley East]|13/02/2002| [13]|Ap #867-859 Sit R...|  39531|(793) 151-6230|134|115|
|     [Evette Dawley]|20/08/1942| [79]|7292 Dictum Av. S...|  47096|(492) 709-6392|139| 78|
|    [Nereida Keeler]|12/05/1973| [52]|Ap #651-8679 Soda...|  10855|(654) 393-5734|120|112|
|     [Valerie Razor]|11/01/1991| [28]|191-103 Integer R...|   8219|(404) 960-3807|143|126|
|    [Camie Berkeley]|18/11/1937| [87]|P.O. Box 887 2508...|  12482|(314) 244-63

## ref_source=File

In [70]:
obfuscator_unique_ref_test = '''Will Perry#PATIENT
John Smith#PATIENT
Marvin MARSHALL#PATIENT
Hubert GROGAN#PATIENT
ALTHEA COLBURN#PATIENT
Kalil AMIN#PATIENT
Inci FOUNTAIN#PATIENT
Jackson WILLE#PATIENT
Jack SANTOS#PATIENT
Mahmood ALBURN#PATIENT
Marnie MELINGTON#PATIENT
Aysha GHAZI#PATIENT
Maryland CODER#PATIENT
Darene GEORGIOUS#PATIENT
Shelly WELLBECK#PATIENT
Min Kun JAE#PATIENT
Thomson THOMAS#PATIENT
Christian SUDDINBURG#PATIENT
Aberdeen#CITY
Louisburg St#STREET
France#LOC
Nick Riviera#DOCTOR
5552312#PHONE
St James Hospital#HOSPITAL
Calle del Libertador#ADDRESS
111#ID
Will#DOCTOR
20#AGE
30#AGE
40#AGE
50#AGE
60#AGE
'''

with open('obfuscator_unique_ref_test.txt', 'w') as f:
  f.write(obfuscator_unique_ref_test)

In [71]:
res= deid_implementor.structured_deidentifier(input_file_path= "deid_structured_data.csv",
                                              ref_source="file", 
                                              columns_dict={'NAME': 'PATIENT', 'AGE': 'AGE'},
                                              columns_seed={"NAME": 23, "AGE": 23}, 
                                              obfuscateRefFile="obfuscator_unique_ref_test.txt")

Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [72]:
res.show()

+------------------+----------+----+--------------------+-------+--------------+---+---+
|              NAME|       DOB| AGE|             ADDRESS|ZIPCODE|           TEL|SBP|DBP|
+------------------+----------+----+--------------------+-------+--------------+---+---+
|   [Inci FOUNTAIN]|04/02/1935|[60]|711-2880 Nulla St...|  69200|(257) 563-7401|101| 42|
|     [Jack SANTOS]|03/10/2009|[30]|P.O. Box 283 8562...|  20620|(372) 587-2335|159|122|
|[Darene GEORGIOUS]|11/01/1921|[30]|5543 Aliquet St. ...|  20783|(717) 450-4729|149| 52|
| [Shelly WELLBECK]|13/02/2002|[40]|Ap #867-859 Sit R...|  39531|(793) 151-6230|134|115|
|   [Hubert GROGAN]|20/08/1942|[40]|7292 Dictum Av. S...|  47096|(492) 709-6392|139| 78|
|      [Kalil AMIN]|12/05/1973|[40]|Ap #651-8679 Soda...|  10855|(654) 393-5734|120|112|
|  [ALTHEA COLBURN]|11/01/1991|[60]|191-103 Integer R...|   8219|(404) 960-3807|143|126|
|  [Thomson THOMAS]|18/11/1937|[60]|P.O. Box 887 2508...|  12482|(314) 244-6306|147| 75|
|     [Jack SANTOS]|1

## shift days

In [73]:
# We can shift n days in the structured deidentification through "days" parameter when the column is a Date.

df = spark.createDataFrame([
            ["Juan García", "13/02/1977", "711 Nulla St.", "140", "673 431234"],
            ["Will Smith", "23/02/1977", "1 Green Avenue.", "140", "+23 (673) 431234"],
            ["Pedro Ximénez", "11/04/1900", "Calle del Libertador, 7", "100", "912 345623"]
        ]).toDF("NAME", "DOB", "ADDRESS", "SBP", "TEL")

df_pd= df.toPandas()
df_pd.to_csv("deid_dayshift_structured_data.csv", index=False)
df_pd.head()

,NAME,DOB,ADDRESS,SBP,TEL
0,Juan García,13/02/1977,711 Nulla St.,140,673 431234
1,Will Smith,23/02/1977,1 Green Avenue.,140,+23 (673) 431234
2,Pedro Ximénez,11/04/1900,"Calle del Libertador, 7",100,912 345623


In [74]:
res= deid_implementor.structured_deidentifier(input_file_path= "deid_dayshift_structured_data.csv",
                                              columns_dict= {"NAME": "ID", "DOB": "DATE"},
                                              columns_seed= {"NAME": 23, "DOB": 23},
                                              ref_source="faker",
                                              shift_days= 5
                                             )

Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [75]:
res.show()

+----------+------------+--------------------+---+----------------+
|      NAME|         DOB|             ADDRESS|SBP|             TEL|
+----------+------------+--------------------+---+----------------+
|[N2649912]|[18/02/1977]|       711 Nulla St.|140|      673 431234|
| [W466004]|[28/02/1977]|     1 Green Avenue.|140|+23 (673) 431234|
| [M403810]|[16/04/1900]|Calle del Liberta...|100|      912 345623|
+----------+------------+--------------------+---+----------------+

